# Introduction to atomman: Load and Dump

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.

Notebook last updated: 2018-04-05
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction

There are a number of built-in converters between the atomman.System class and other representations of atomic configurations.  
 
 - **load** creates a System object from another representation.
 
 - **dump** converts a System to another representation.
 
### 1.1. Load details

Content can be loaded using the atomman.load() function. 

Parameters

- **style** (*str*) indicates the format of the content being loaded.

- **content** (*any*) the content to be loaded.  For text formats, can be a str of the content, a file path or a file-like object.

- **\*\*kwargs** (*any*) any extra style-specific keyword arguments.

Alternatively, there is a load_STYLE() function for each style that takes the content and kwargs.  The underlying code is found in the atomman/load directory.

### 1.2. Dump details

A system can be dumped to another format using the system.dump() method. 

Parameters

- **style** (*str*) indicates the format of the content to dump to.

- **\*\*kwargs** (*any*) any extra style-specific keyword arguments.

Alternatively, there are atomman.dump() and atomman.dump_style() functions that can be used to dump a system to the different style formats. The dump() function takes style, system and kwargs as parameters, while the dump_style() functions take system and kwargs. The underlying code is found in the atomman/dump directory.


**Library Imports**

In [25]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)

# http://www.numpy.org/
import numpy as np

import atomman as am
import atomman.unitconvert as uc

Generate test system information (CsCl)

In [26]:
# Generate box
alat = uc.set_in_units(3.2, 'angstrom')
box = am.Box(a=alat, b=alat, c=alat)

# Generate atoms with atype, pos, charge, and stress properties
atype = [1, 2]
pos = [[0,0,0], [0.5, 0.5, 0.5]]
charge = uc.set_in_units([1, -1], 'e')
stress = uc.set_in_units(np.zeros((2, 3, 3)), 'MPa')
atoms = am.Atoms(pos=pos, atype=atype, charge=charge, stress=stress)

# Build system from box and atoms, and scale atoms
system = am.System(atoms=atoms, box=box, scale=True, symbols=['Cs', 'Cl'])

# Print system information
print(system)
system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = (u'Cs', u'Cl')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2],charge
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1.6,1.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


## 2. system_model style

The system_model style provides a complete structured record of all values with proper units that is compatible with JSON or XML databases.  The downside is that the representation is not as efficient or convenient as tabular formats for large numbers of atoms and atomic properties. 

### 2.1. dump('system_model')

Keyword Parameters

- **box_unit** (*str, optional*) Length unit to use for the box. Default value is 'angstrom'.

- **symbols** (*list, optional*) list of atom-model symbols corresponding to the atom types.  Will use system.symbols if not given.

- **elements** (*list, optional*) list of element tags corresponding to the atom types.

- **prop_units** (*dict, optional*) dictionary where the keys are the property keys to include, and the values are units to use. If not given, only the positions in scaled units are included.

- **a_std** (*float, optional*) Standard deviation of a lattice constant to include if available.

- **b_std** (*float, optional*) Standard deviation of b lattice constant to include if available.

- **c_std** (*float, optional*) Standard deviation of c lattice constant to include if available.

Returns

- **model** (*DataModelDict*) A data model of the system.

In [27]:
model = system.dump('system_model',  box_unit='angstrom', prop_units={'charge': 'e', 'stress': 'GPa'})
print('JSON system model:')
print(model.json())
print()

print('XML system model:')
print(model.xml())

JSON system model:
{"atomic-system": {"cell": {"cubic": {"a": {"value": 3.2000000000000006, "unit": "angstrom"}}}, "atom": [{"component": 1, "symbol": "Cs", "position": {"value": [0.0, 0.0, 0.0], "unit": "scaled"}, "property": [{"name": "charge", "value": 1.0, "unit": "e"}, {"name": "stress", "value": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "shape": [3, 3], "unit": "GPa"}]}, {"component": 2, "symbol": "Cl", "position": {"value": [0.5, 0.5, 0.5], "unit": "scaled"}, "property": [{"name": "charge", "value": -1.0, "unit": "e"}, {"name": "stress", "value": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "shape": [3, 3], "unit": "GPa"}]}]}}

XML system model:
<?xml version="1.0" encoding="utf-8"?>
<atomic-system><cell><cubic><a><value>3.2000000000000006</value><unit>angstrom</unit></a></cubic></cell><atom><component>1</component><symbol>Cs</symbol><position><value>0.0</value><value>0.0</value><value>0.0</value><unit>scaled</unit></position><property><name>charge</name><value>1.0</value

### 2.2. load('system_model')

Parameters

- **model** (*str, file-like object or DataModelDict*) The data model to read.

- **key** (*str, optional*) The key identifying the root element for the system definition. Default value is 'atomic-system'.

- **index** (*int, optional*) If the full model has multiple key entries, the index specifies which to access.  Default value is 0 (first, or only entry).

Returns

- **system** (*atomman.System*) The system object associated with the data model.


In [28]:
model_system = am.load('system_model', model)
print(model_system)
model_system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = (u'Cs', u'Cl')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2],charge
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1.6,1.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


## 3. poscar style

The POSCAR format is a simple tabular format used by VASP that captures box, element types and position.  It is limited in that it cannot contain extra per-atom properties.

### 3.1. dump('poscar')

Parameters

- **fname** (*str, optional*) Path to file where POSCAR content is saved to.  If not given, a str of the content will be returned.

- **header** (*str, optional*) The comment line to place at the top of the file. Default value is ''.

- **symbols** (*list of str, optional*) List of the element symbols that correspond to the atom types.  If not given, will use system.symbols if set, otherwise no element content will be included.

- **style** (*str, optional*) The poscar coordinate style.  Default value is 'direct'.

- **box_scale** (*float, optional*) A universal scaling constant applied to the box vectors. Default value is 1.0.

- **xf** (*str, optional*) c-style format for printing the floating point numbers. Default value is '%.13e'.
        
Returns

- **poscar_str** (*str*) String of the poscar object (only returned if fname is not given).

In [29]:
poscar = system.dump('poscar', header='Test POSCAR content for CsCl')
print(poscar)

Test POSCAR content for CsCl
1.0000000000000e+00
3.2000000000000e+00 0.0000000000000e+00 0.0000000000000e+00
0.0000000000000e+00 3.2000000000000e+00 0.0000000000000e+00
0.0000000000000e+00 0.0000000000000e+00 3.2000000000000e+00
Cs Cl
1 1 
direct
0.0000000000000e+00 0.0000000000000e+00 0.0000000000000e+00
5.0000000000000e-01 5.0000000000000e-01 5.0000000000000e-01


### 3.2. load('poscar')

Parameters

- **poscar** (*str or file-like object*) The POSCAR content to read.

- **prop** (*dict, optional*) Dictionary containing any extra per-atom properties to include.
    
Returns

- **system** (*atomman.System*) The system object associated with the data model.

In [30]:
# Extract extra per-atom properties from currently loaded system
prop = {}
for p in system.atoms_prop():
    if p != 'atype' and p != 'pos':
        prop[p] = system.atoms_prop(key=p)

# Load poscar and pass in prop dictionary
poscar_system = am.load('poscar', poscar, prop=prop)
print(poscar_system)
poscar_system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = (u'Cs', u'Cl')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2],charge
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1.6,1.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


## 4. cif style

The Crystallographic Information File (CIF) format is a standard for representing crystallographic information and is used by different crystal databases.  Loading CIF files requires that the [diffpy.Structure](https://github.com/diffpy) package be installed (Not yet Python3 compatible as of Feb 7 2018).

### 4.1. load('cif')

Parameters

- **cif** (*str or file-like object*) The cif content to read.
        
Returns

- **system** (*atomman.System*) An atomman representation of a system.

In [31]:
# CIF file taken from Crystallography Open Database (COD): http://www.crystallography.net/
cif = """#------------------------------------------------------------------------------
#$Date: 2016-02-13 21:28:24 +0200 (Sat, 13 Feb 2016) $
#$Revision: 176429 $
#$URL: svn://www.crystallography.net/cod/cif/1/54/12/1541266.cif $
#------------------------------------------------------------------------------
#
# This file is available in the Crystallography Open Database (COD),
# http://www.crystallography.net/
#
# All data on this site have been placed in the public domain by the
# contributors.
#
data_1541266
loop_
_publ_author_name
'Mueller, M. H.'
_publ_section_title
;
 The lattice parameter of tantalum
;
_journal_issue                   8
_journal_name_full               'Scripta Metallurgica'
_journal_page_first              693
_journal_page_last               693
_journal_paper_doi               10.1016/0036-9748(77)90141-7
_journal_volume                  11
_journal_year                    1977
_chemical_formula_sum            Ta
_chemical_name_systematic        Ta
_space_group_IT_number           229
_symmetry_space_group_name_Hall  '-I 4 2 3'
_symmetry_space_group_name_H-M   'I m -3 m'
_cell_angle_alpha                90
_cell_angle_beta                 90
_cell_angle_gamma                90
_cell_formula_units_Z            2
_cell_length_a                   3.30256
_cell_length_b                   3.30256
_cell_length_c                   3.30256
_cell_volume                     36.021
_citation_journal_id_ASTM        SCRMBU
_cod_data_source_file            Mueller_SCRMBU_1977_100.cif
_cod_data_source_block           Ta1
_cod_original_cell_volume        36.0207
_cod_original_formula_sum        Ta1
_cod_database_code               1541266
loop_
_symmetry_equiv_pos_as_xyz
x,y,z
-y,x,z
-x,-y,z
y,-x,z
x,-y,-z
y,x,-z
-x,y,-z
-y,-x,-z
z,x,y
-x,z,y
-z,-x,y
x,-z,y
z,-x,-y
x,z,-y
-z,x,-y
-x,-z,-y
y,z,x
y,-z,-x
z,y,-x
-y,z,-x
-z,-y,-x
-y,-z,x
z,-y,x
-z,y,x
-x,-y,-z
y,-x,-z
x,y,-z
-y,x,-z
-x,y,z
-y,-x,z
x,-y,z
y,x,z
-z,-x,-y
x,-z,-y
z,x,-y
-x,z,-y
-z,x,y
-x,-z,y
z,-x,y
x,z,y
-y,-z,-x
-y,z,x
-z,-y,x
y,-z,x
z,y,x
y,z,-x
-z,y,-x
z,-y,-x
x+1/2,y+1/2,z+1/2
-y+1/2,x+1/2,z+1/2
-x+1/2,-y+1/2,z+1/2
y+1/2,-x+1/2,z+1/2
x+1/2,-y+1/2,-z+1/2
y+1/2,x+1/2,-z+1/2
-x+1/2,y+1/2,-z+1/2
-y+1/2,-x+1/2,-z+1/2
z+1/2,x+1/2,y+1/2
-x+1/2,z+1/2,y+1/2
-z+1/2,-x+1/2,y+1/2
x+1/2,-z+1/2,y+1/2
z+1/2,-x+1/2,-y+1/2
x+1/2,z+1/2,-y+1/2
-z+1/2,x+1/2,-y+1/2
-x+1/2,-z+1/2,-y+1/2
y+1/2,z+1/2,x+1/2
y+1/2,-z+1/2,-x+1/2
z+1/2,y+1/2,-x+1/2
-y+1/2,z+1/2,-x+1/2
-z+1/2,-y+1/2,-x+1/2
-y+1/2,-z+1/2,x+1/2
z+1/2,-y+1/2,x+1/2
-z+1/2,y+1/2,x+1/2
-x+1/2,-y+1/2,-z+1/2
y+1/2,-x+1/2,-z+1/2
x+1/2,y+1/2,-z+1/2
-y+1/2,x+1/2,-z+1/2
-x+1/2,y+1/2,z+1/2
-y+1/2,-x+1/2,z+1/2
x+1/2,-y+1/2,z+1/2
y+1/2,x+1/2,z+1/2
-z+1/2,-x+1/2,-y+1/2
x+1/2,-z+1/2,-y+1/2
z+1/2,x+1/2,-y+1/2
-x+1/2,z+1/2,-y+1/2
-z+1/2,x+1/2,y+1/2
-x+1/2,-z+1/2,y+1/2
z+1/2,-x+1/2,y+1/2
x+1/2,z+1/2,y+1/2
-y+1/2,-z+1/2,-x+1/2
-y+1/2,z+1/2,x+1/2
-z+1/2,-y+1/2,x+1/2
y+1/2,-z+1/2,x+1/2
z+1/2,y+1/2,x+1/2
y+1/2,z+1/2,-x+1/2
-z+1/2,y+1/2,-x+1/2
z+1/2,-y+1/2,-x+1/2
loop_
_atom_site_label
_atom_site_type_symbol
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
_atom_site_occupancy
_atom_site_U_iso_or_equiv
Ta1 Ta 0 0 0 1 0.0
"""

In [32]:
try:
    cif_system = am.load('cif', cif)
except AssertionError as e:
    print('AssertionError:', e)
else:
    print(cif_system)
    print(cif_system.atoms_df())

avect =  [ 3.303,  0.000,  0.000]
bvect =  [ 0.000,  3.303,  0.000]
cvect =  [ 0.000,  0.000,  3.303]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 1
symbols = ('Ta',)
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   1.651 |   1.651 |   1.651
   atype   pos[0]   pos[1]   pos[2]
0      1  0.00000  0.00000  0.00000
1      1  1.65128  1.65128  1.65128


## 5. table style

The table style returns the atomic information in a generic table format.

### 5.1 dump('table')

Parameters

- **fp** (*str or file-like object, optional*) File path or file-like object to write the table to.  If not given, then the table is returned as a string.

- **prop_name** (*list, optional*) The Atoms properties to include.  Must be given if prop_info is not.

- **table_name** (*list, optional*) The table column name(s) that correspond to each prop_name.  If not given, the table_name values will be based on the prop_name values.

- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be inferred from the length of each table_name value.

- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).

- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given. Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.

- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.

- **header** (*bool, optional*) Flag indicating whether to include the column names in the outputted table.  Default value is False (no column names).

- **float_format** (*str, optional*) c-style formatting string for floating point values.  Default value is '%.13f'.

- **return_prop_info** (*bool, optional*) Flag indicating if the filled-in prop_info is to be returned.  Having this allows for 1:1 load/dump conversions.  Default value is False (prop_info is not returned).

Returns

- (*str*) The generated data table.  Only returned if fp is None.

In [37]:
table, prop_info = system.dump('table', float_format='%.5f', header=True, return_prop_info=True)
print(table)

atype pos[0] pos[1] pos[2] stress[0][0] stress[0][1] stress[0][2] stress[1][0] stress[1][1] stress[1][2] stress[2][0] stress[2][1] stress[2][2] charge
1 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 1.00000
2 1.60000 1.60000 1.60000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 -1.00000



The prop_info returned is json describing the conversion between atoms property and table representations of the data.

In [38]:
# Show the generated prop_info conversion info
for pinfo in prop_info:
    print(pinfo)

{u'dtype': None, u'prop_name': u'atype', u'table_name': [u'atype'], u'shape': (), u'unit': None}
{u'dtype': None, u'prop_name': u'pos', u'table_name': [u'pos[0]', u'pos[1]', u'pos[2]'], u'shape': (3L,), u'unit': None}
{u'dtype': None, u'prop_name': u'stress', u'table_name': [u'stress[0][0]', u'stress[0][1]', u'stress[0][2]', u'stress[1][0]', u'stress[1][1]', u'stress[1][2]', u'stress[2][0]', u'stress[2][1]', u'stress[2][2]'], u'shape': (3L, 3L), u'unit': None}
{u'dtype': None, u'prop_name': u'charge', u'table_name': [u'charge'], u'shape': (), u'unit': None}


### 5.2 load('table')

Parameters

- **table** (*str or file-like object*) The table content, file path or file-like object containing the content to read.

- **box** (*atomman.Box*) The atomic box to use when generating a System around the data.

- **symbols** (*tuple, optional*) Allows the list of element symbols to be assigned during loading.

- **system** (*atomman.System, optional*) The atomic system to load the values to.  If not given, a new system will be constructed.

- **prop_name** (*list, optional*) The Atoms properties to generate.  Must be given if prop_info is not.

- **table_name** (*list, optional*) The table column name(s) that correspond to each prop_name.  If not given, the table_name values will be based on the prop_name values.

- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be inferred from the length of each table_name value.

- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).

- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given.  Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.

- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.

- **skiprows** (*int*) Number of rows to skip before reading the data.

- **nrows** (*int*) Number of rows of data to read.

Returns

- (*atomman.System*) The generated system.

In [39]:
# Pass table with box and prop_info
table_system = am.load('table', table, box=system.box, symbols=['Cs', 'Cl'],
                       skiprows=1, prop_info=prop_info)

print(table_system)
table_system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = (u'Cs', u'Cl')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2],charge
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1.6,1.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


In [40]:
# Using the generated prop_info from dump makes the resulting properties have the right shape.
print(table_system.atoms.stress[0])

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## 6. atom_data style

The atom_data format is the atomic data files used by LAMMPS for importing initial configurations. 

### 6.1. dump('atom_data')

Parameters
        
- **f** (*str or file-like object, optional*) File path or file-like object to write the content to.  If not given, then the content is returned as a str.
        
- **atom_style** (*str, optional*) The LAMMPS atom_style option associated with the data file.  Default value is 'atomic'.
        
- **units** (*str, optional*) The LAMMPS units option associated with the data file. Default value is 'metal'.
        
- **float_format** (*str, optional*) c-style formatting string for floating point values.  Default value is '%.13f'.

- **return_info** (*bool, optional*) Indicates if the LAMMPS command lines associated with reading in the file are to be returned as a str.  Default value is True.
      
Returns

- **content** (*str*) The data file contents (returned if f is not given).

- **read_info** (*str*) The LAMMPS input command lines to read the created data file in (returned if return_info is True).

In [41]:
data, read_info = system.dump('atom_data')
print(data)


2 atoms
2 atom types
0.0000000000000 3.2000000000000 xlo xhi
0.0000000000000 3.2000000000000 ylo yhi
0.0000000000000 3.2000000000000 zlo zhi

Atoms

1 1 0.0000000000000 0.0000000000000 0.0000000000000
2 2 1.6000000000000 1.6000000000000 1.6000000000000



In [42]:
# read_info contains the LAMMPS command lines for reading the data content
print(read_info)

# Script and atom data file prepared by atomman package

units metal
atom_style atomic
boundary p p p 



### 6.2. load('atom_data')

Parameters

- **data** (*str or file-like object*) The atom data content to read.  Can be str content, path name, or open file-like object.

- **pbc** (*list of bool*) Three boolean values indicating which System directions are periodic.  Default value is (True, True, True).
    
- **symbols** (*tuple, optional*) Allows the list of element symbols to be assigned during loading.        
- **atom_style** (*str, optional*) The LAMMPS atom_style option associated with the data file.  Default value is 'atomic'.
        
- **units** (*str, optional*) The LAMMPS units option associated with the data file. Default value is 'metal'.
      
Returns

- (*atomman.System*) The corresponding system.  Note all property values will be automatically converted to atomman.unitconvert's set working units.

In [15]:
# Read in data file
data_system = am.load('atom_data', data, symbols=['Cs', 'Cl'])

# Note that only atype and pos are retrieved as that is all the data file format held
print(data_system)
data_system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = (u'Cs', u'Cl')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2]
0,1,0.0,0.0,0.0
1,2,1.6,1.6,1.6


## 7. atom_dump style

The atom_dump format is the LAMMPS dump files containing atomic coordinates and other per-atom properties.

### 7.1 dump('atom_dump')

Parameters

- **f** (*str or file-like object, optional*) File path or file-like object to write the table to.  If not given, then the table is returned as a string.

- **lammps_units** (*str, optional*) The LAMMPS units option associated with the table values.  This is used for the box dimensions and default units for standard dump properties (not compute/fix definitions). 

- **scale** (*bool, optional*) Flag indicating if atom positions are to be scaled relative to the box (True) or given in absolute Cartesian values (False, default).

- **prop_name** (*list, optional*) The Atoms properties to include.  If neither prop_name or prop_info are given, all system properties will be included.

- **table_name** (*list, optional*) The dump table column name(s) that correspond to each prop_name.  If not given, the table_name values will be based on the prop_name and shape values.

- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be inferred from the length of each table_name value.

- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).

- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given. Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.

- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.

- **header** (*bool, optional*) Flag indicating whether to include the column names in the outputted table.  Default value is False (no column names).

- **float_format** (*str, optional*) c-style formatting string for floating point values.  Default value is '%.13f'.

- **return_prop_info** (*bool, optional*) Flag indicating if the filled-in prop_info is to be returned.  Having this allows for 1:1 load/dump conversions.  Default value is False (prop_info is not returned).

Returns

- **content** (*str*) The generated atom_data content.  Only returned if f is None.

- **prop_info** (*list of dict*) The full prop_info detailing the property-table conversion.

In [43]:
atom_dump, prop_info = system.dump('atom_dump', return_prop_info=True)

print(atom_dump)

ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
2
ITEM: BOX BOUNDS pp pp pp
0.0000000000000 3.2000000000000
0.0000000000000 3.2000000000000
0.0000000000000 3.2000000000000
ITEM: ATOMS id type x y z stress[0][0] stress[0][1] stress[0][2] stress[1][0] stress[1][1] stress[1][2] stress[2][0] stress[2][1] stress[2][2] q
1 1 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 1.0000000000000
2 2 1.6000000000000 1.6000000000000 1.6000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 -1.0000000000000



In [44]:
# Show the generated prop_info conversion info
for pinfo in prop_info:
    print(pinfo)

{u'dtype': None, u'prop_name': u'a_id', u'table_name': [u'id'], u'shape': (), u'unit': None}
{u'dtype': None, u'prop_name': u'atype', u'table_name': [u'type'], u'shape': (), u'unit': None}
{u'dtype': None, u'prop_name': u'pos', u'table_name': [u'x', u'y', u'z'], u'shape': (3L,), u'unit': u'angstrom'}
{u'dtype': None, u'prop_name': u'stress', u'table_name': [u'stress[0][0]', u'stress[0][1]', u'stress[0][2]', u'stress[1][0]', u'stress[1][1]', u'stress[1][2]', u'stress[2][0]', u'stress[2][1]', u'stress[2][2]'], u'shape': (3L, 3L), u'unit': None}
{u'dtype': None, u'prop_name': u'charge', u'table_name': [u'q'], u'shape': (), u'unit': u'e'}


### 7.2 load('atom_dump')

Parameters

- **data** (*str or file-like object*) The content, file path or file-like object containing the content to read.

- **symbols** (*tuple, optional*) Allows the list of element symbols to be assigned during loading.

- **lammps_units** (*str*) The LAMMPS units option associated with the parameters.  Default value is 'metal'.

- **prop_name** (*list, optional*) The Atoms properties to generate.
         
- **table_name** (*list, optional*) The table column name(s) that correspond to each prop_name.  If prop_name, table_name and prop_info are not given, prop_name and table_name will be read in from data.
        
- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be taken from standard LAMMPS parameter names, or left at () for direct property-table conversion.
        
- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).
        
- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given.  Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.
        
- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.
        
- **return_prop_info** (*bool, optional*) Flag indicating if the full prop_info is to be returned.  Default value is False.
        
Returns

- **system** (*atomman.System*) The generated system.

- **prop_info** (*list of dict*) The full prop_info detailing the property-table conversion. Returned if return_prop_info is True.

In [45]:
# Read in dump file
atom_dump_system = am.load('atom_dump', atom_dump, symbols=['Cs', 'Cl'])

print(atom_dump_system)
atom_dump_system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = (u'Cs', u'Cl')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2],charge
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1.6,1.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


## 8. ase_Atoms style

There is also built-in converters between the alternate Python atomistic package of [ase](https://wiki.fysik.dtu.dk/ase/).  Requires that ase be installed.

In [46]:
try:
    import ase
    has_ase = True
except:
    print('ase not installed')
    has_ase = False

### 8.1. dump('ase_Atoms')

Parameters

- **symbols** (*list*) A list of unique elemental symbols to pair with system's atypes.  If not given, will try to use system.symbols.
    
Returns

- **aseatoms** (*ase.Atoms*) An ase representation of a collection of atoms.

- **prop** (*dict*) Dictionary containing any extra per-atom properties to include.

In [47]:
if has_ase:
    aseatoms, prop = system.dump('ase_Atoms', return_prop=True)
else:
    print('ase not installed')
    aseatoms, prop = None
print(aseatoms)
print(prop)

Atoms(symbols='CsCl', pbc=True, cell=[3.2, 3.2, 3.2])
{u'stress': array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]]), u'charge': array([ 1., -1.])}


### 8.2. load('ase_Atoms')

Parameters

- **aseatoms** (*ase.Atoms*) An ase representation of a collection of atoms.

- **prop** (*dict, optional*) Dictionary containing any extra per-atom properties to include.
        
Returns

- **system** (*atomman.System*) A atomman representation of a system.


In [48]:
if has_ase:
    system = am.load('ase_Atoms', aseatoms, prop=prop)

else:
    print('ase not installed (elements and system unchanged from above)')
    
print(system)
system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = ('Cl', 'Cs')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       2 |   0.000 |   0.000 |   0.000
      1 |       1 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2],charge
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1.6,1.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


## 9. pymatgen_Structure style

There is also built-in converters between the alternate Python atomistic package of [pymatgen](http://pymatgen.org/).  Requires that pymatgen be installed. (No longer supports Python 2).

In [49]:
try:
    import pymatgen as pmg
    has_pmg = True
except:
    print('pymatgen not installed')
    has_pmg = False

pymatgen not installed


### 9.1. dump('pymatgen_Structure')

Parameters

- **symbols** (*list*) A list of unique elemental symbols to pair with system's atypes.  Will try to use system.symbols if not given.
    
Returns

- **structure** (*pymatgen.Structure*) A pymatgen representation of a structure.

In [50]:
if has_pmg:
    structure = system.dump('pymatgen_Structure')
else:
    print('pymatgen not installed')
    structure = None
    
print(structure)

pymatgen not installed
None


### 9.2. load('pymatgen_Structure')

Parameters

- **structure** (*pymatgen.Structure*) A pymatgen representation of a structure.
        
Returns

- **system** (*atomman.System*) A atomman representation of a system.

In [51]:
if has_pmg:
    system = am.load('pymatgen_Structure', structure)
else:
    print('pymatgen not installed (elements and system unchanged from above)')
    
print(system)
system.atoms_df()

pymatgen not installed (elements and system unchanged from above)
avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = ('Cl', 'Cs')
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       2 |   0.000 |   0.000 |   0.000
      1 |       1 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2],charge
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1.6,1.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
